# Imports

In [1]:
import conversions
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import models
from torchvision.models import mobilenet_v2
from tqdm import tqdm
from torchsummary import summary
import time

# Loading the dataset

In [2]:
# Fix the seed to ensure reproducibility
torch.manual_seed(42)

# Data augmentation for training (applied only to the train dataset)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize to MobileNet input size
    transforms.RandomHorizontalFlip(), # Flip horizontally
    transforms.RandomRotation(15), # Random rotation 15 degress
    transforms.ToTensor(), # Convert to tensor
    transforms.Normalize((0.5,), (0.5,)) # Normalize (mean=0.5, std=0.5)
])

# No augmentation for validation/test (only resizing and normalization)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# Download CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Split the test_data into test (6k) and dev (4k)
test_size = 6000
dev_size = 4000
test_dataset, dev_dataset = random_split(test_data, [test_size, dev_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Print dataset sizes
print(f"Train set size: {len(train_dataset)}")
print(f"Dev set size: {len(dev_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Train set size: 50000
Dev set size: 4000
Test set size: 6000


# Defining the model architecture 📐👷🏻‍♀️ and loading the pre-trained weights 🧠

In [3]:
# Load the pre-trained MobileNet model
base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

# Unfreeze some of the top layers
for param in base_model.features[:-10].parameters():
    param.requires_grad = False

# Modify the classifier for CIFAR-10 (10 classes)
class MobileNetV2(nn.Module):
    def __init__(self, base_model):
        super(MobileNetV2, self).__init__()
        self.features = base_model.features
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Linear(1280, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.Linear(64, 10)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = MobileNetV2(base_model)

In [4]:
model_weights_path = './pytorch_models/mobilenet_sparse.pth'
# load the full model
model.load_state_dict(torch.load(model_weights_path))
# Set the model to evaluation mode
model.eval()

print("Model weights loaded successfully!")

RuntimeError: Error(s) in loading state_dict for MobileNetV2:
	While copying the parameter named "features.0.0.weight", whose dimensions in the model are torch.Size([32, 3, 3, 3]) and whose dimensions in the checkpoint are torch.Size([32, 3, 3, 3]), an exception occurred : ('copy_() between dense and sparse Tensors is not implemented! Found self type = torch.FloatTensor and src type = torch.cuda.sparse.FloatTensor',).
	While copying the parameter named "features.1.conv.0.0.weight", whose dimensions in the model are torch.Size([32, 1, 3, 3]) and whose dimensions in the checkpoint are torch.Size([32, 1, 3, 3]), an exception occurred : ('copy_() between dense and sparse Tensors is not implemented! Found self type = torch.FloatTensor and src type = torch.cuda.sparse.FloatTensor',).
	While copying the parameter named "features.1.conv.1.weight", whose dimensions in the model are torch.Size([16, 32, 1, 1]) and whose dimensions in the checkpoint are torch.Size([16, 32, 1, 1]), an exception occ

In [17]:
# Ensure the model is on the correct device (cuda or cpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

# Test conversion to onnx function

In [19]:
conversions.torch2onnx("./pruned_mobilenet_l2.onnx", model, device)
conversions.check_onnx_model("./pruned_mobilenet_l2.onnx")

Model successfully exported to: ./pruned_mobilenet_l2.onnx
Model successfully checked: ./pruned_mobilenet_l2.onnx


# Test conversion from onnx to TRT

In [2]:
conversions.onnx2trt("./pruned_mobilenet_l2.onnx", "./pruned_mobilenet_l2_fp16.trt", fp_16=True)
conversions.onnx2trt("./pruned_mobilenet_l2.onnx", "./pruned_mobilenet_l2.trt", fp_16=False)

FP16 supported, enabling FP16 optimization.
Building TensorRT engine. This might take a while...
TensorRT engine saved to ./pruned_mobilenet_l2_fp16.trt
Building TensorRT engine. This might take a while...
TensorRT engine saved to ./pruned_mobilenet_l2.trt


In [3]:
conversions.trt_inference("./trt_models/pruned_mobilenet_l2_fp16.trt", test_loader)

Running inference on the test dataset...


100%|██████████| 94/94 [00:12<00:00,  7.83it/s]


Test Accuracy: 73.95%
Total Inference Time: 12.02 seconds


(73.95, 12.01703429222107)